# Iterative Prompt Development
In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install -r "/content/drive/MyDrive/LLM/Groq/requirements.txt"

In [ ]:
from groq import Groq
import os  ## step-3
# Read and set the environment variable from the .bashrc file
with open('/content/drive/MyDrive/LLM/Groq/.bashrc') as file:
    for line in file:
        if line.startswith('export '):
            var, value = line[len('export '):].strip().split('=')
            os.environ[var] = value

# Verify that the environment variable is set
!echo $GROQ_API_KEY

In [ ]:

client = Groq(api_key = os.getenv('GROQ_API_KEY'))
model = 'mixtral-8x7b-32768'

In [ ]:
def get_completion(prompt, model="llama3-70b-8192"):
    messages = [{"role": "user", "content": prompt}]  # Include the word 'json' in the messages parameter
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

## Generate a marketing product description from a product fact sheet

In [ ]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100)
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black,
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Here is a product description based on the technical specifications:

**Introducing the Mid-Century Inspired Office Chair**

Elevate your workspace with our stunning mid-century inspired office chair, part of a beautiful family of office furniture designed to bring style and sophistication to your home or business setting. With a range of shell color and base finish options, you can customize your chair to fit your unique style.

Choose from 10 fabric and 6 leather upholstery options, or opt for a sleek plastic back and front upholstery. The base finish options include stainless steel, matte black, gloss white, or chrome, ensuring a perfect match with your office decor. Plus, you can choose to add armrests for extra comfort.

This chair is built to last, with a sturdy 5-wheel plastic coated aluminum base and pneumatic chair adjust for easy height adjustment. The seat is crafted with high-density foam for optimal comfort and support.

With a range of options to suit your needs, includin

## Issue 1: The text is too long
- Limit the number of words/sentences/characters.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Here is a product description based on the technical specifications:

"Elevate your workspace with our mid-century inspired office chair, featuring a sturdy aluminum base, adjustable height, and comfortable upholstery options in 10 fabrics and 6 leathers. Customize to fit your style with various shell colors, base finishes, and armrest options."


In [ ]:
len(response.split())

50

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Here is a product description based on the technical specifications:

"Constructed with a durable cast aluminum shell base glider featuring a modified nylon PA6/PA66 coating, this mid-century inspired office chair boasts a sturdy 5-wheel plastic coated aluminum base and HD36 foam seat. Available with various shell color and base finish options, including stainless steel, matte black, gloss white, or chrome."


In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Here is a product description based on the technical specifications:

"This mid-century inspired office chair features a durable cast aluminum shell base glider with a modified nylon PA6/PA66 coating, paired with a HD36 foam seat. The 5-wheel plastic coated aluminum base provides stability, while the pneumatic chair adjust allows for easy height adjustment. Available in various shell colors and base finishes, including stainless steel, matte black, gloss white, and chrome."

Product IDs: SWC-100, SWC-110


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

After the description, include a table that gives the
product's dimensions. The table should have two columns.
In the first column include the name of the dimension.
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website.
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

Here is the product description and dimensions table in HTML format:

<div>
This mid-century inspired office chair is part of a beautiful family of furniture, offering a range of customization options to suit your needs. The chair features a sturdy 5-wheel plastic coated aluminum base and a pneumatic chair adjust for easy raise/lower action. The shell is constructed from cast aluminum with a modified nylon PA6/PA66 coating, with a thickness of 10 mm. The seat is made from HD36 foam, available in two densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3). The chair is available with or without armrests, and with a range of base finish options including stainless steel, matte black, gloss white, or chrome.

Product IDs: SWC-100, SWC-110

</div>

<table>
  <caption>Product Dimensions</caption>
  <tr><td>WIDTH</td><td>20.87”</td></tr>
  <tr><td>DEPTH</td><td>20.08”</td></tr>
  <tr><td>HEIGHT</td><td>31.50”</td></tr>
  <tr><td>SEAT HEIGHT</td><td>17.32”</td></tr>
  <tr><td>SEAT DEPTH</td><td>1

## Load Python libraries to view HTML

In [ ]:
from IPython.display import display, HTML

In [ ]:
display(HTML(response))

WIDTH,20.87”
DEPTH,20.08”
HEIGHT,31.50”
SEAT HEIGHT,17.32”
SEAT DEPTH,16.14”


## Try experimenting on your own!